In [1]:
# Dependencies
import requests
import json
import pandas as pd

from pprint import pprint

# Google developer API key
import sys
sys.path.append('../..')
from config import gkey


file = pd.read_csv("winedata_edited.csv")

file.head(30)

,Unnamed: 0,Title,Variety,Year,Price,Grade,Winery,Province,Region,Country,Taster,Taster_Twitter,Description
0,1,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,2011,15.0,87,Quinta dos Avidagos,Douro,NaN,Portugal,Roger Voss,@vossroger,"This is ripe and fruity, a wine that is smooth..."
1,2,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,2013,14.0,87,Rainstorm,Oregon,Willamette Valley,US,Paul Gregutt,@paulgwine,"Tart and snappy, the flavors of lime flesh and..."
2,3,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,2013,13.0,87,St. Julian,Michigan,Lake Michigan Shore,US,Alexander Peartree,NaN,"Pineapple rind, lemon pith and orange blossom ..."
3,4,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,2012,65.0,87,Sweet Cheeks,Oregon,Willamette Valley,US,Paul Gregutt,@paulgwine,"Much like the regular bottling from 2012, this..."
4,5,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,2011,15.0,87,Tandem,Northern Spain,Navarra,Spain,Michael Schachner,@wineschach,Blackberry and raspberry aromas show a typical...
5,6,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,2013,16.0,87,Terre di Giurfo,Sicily & Sardinia,Vittoria,Italy,Kerin O’Keefe,@kerinokeefe,"Here's a bright, informal red that opens with ..."
6,7,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,2012,24.0,87,Trimbach,Alsace,Alsace,France,Roger Voss,@vossroger,This dry and restrained wine offers spice in p...
7,8,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,2013,12.0,87,Heinz Eifel,Rheinhessen,NaN,Germany,Anna Lee C. Iijima,NaN,Savory dried thyme notes accent sunnier flavor...
8,9,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,2012,27.0,87,Jean-Baptiste Adam,Alsace,Alsace,France,Roger Voss,@vossroger,This has great depth of flavor with its fresh ...
9,10,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,2011,19.0,87,Kirkland Signature,California,Napa Valley,US,Virginie Boone,@vboone,"Soft, supple plum envelopes an oaky structure ..."


In [2]:
#drop lines with too vague location
file = file.loc[file["Province"] != "America"]

In [ ]:
file["Location"] = file["Province"] + " " + file["Country"]

In [6]:
province = []

for l in file["Location"]:
    if l not in province:
        province.append(l)

In [7]:
# Target location

province_response =[]

# Run API calls for each winery in list
for prov in province:

    target_location = prov 
    
    # Build the endpoint URL
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_location, gkey)
    
    # Run a request to endpoint and convert result to json
    geo_data = requests.get(target_url).json()
   
    # Add response to our list
    province_response.append(geo_data)        

In [9]:
# Collect and store latitude and longitude for each province

lat = []
lng = []
#county = []

i=0

# Loop through wineries and append coordinate lists
for response in province_response:
    try:
        latitude = province_response[i]["results"][0]["geometry"]["location"]["lat"]
        longitude = province_response[i]["results"][0]["geometry"]["location"]["lng"]
  #      county_name = province_response[i]["results"][0]["address_components"][1]["long_name"]

        lat.append(latitude)
        lng.append(longitude)
  #      county.append(county_name)
        
        i = i+1
    except:
        lat.append("lat error")
        lng.append("lng error")
 #       county.append("county error")
        
        i = i+1

In [16]:
province_df= pd.DataFrame({"Location" : province,
                          "Latitude" : lat,
                          "Longitude" : lng
                          })

In [14]:
# Drop rows without location data
province_df = province_df.loc[province_df["Latitude"] != "lat error"]

In [ ]:
# Add coordinates to primary dataframe by merging
complete_df = pd.merge(file, province_df, on="Location")

In [18]:
for index, row in complete_df.iterrows():
#blush wines
    if "Zinfandel" in row["Variety"]:
        complete_df.loc[index,"Type"] = "Blush"
    elif "Rosa" in row["Variety"]:
        complete_df.loc[index,"Type"] = "Blush"
    elif "Rosé" in row["Variety"]:
        complete_df.loc[index,"Type"] = "Blush"
#sparkling wines
    elif "Sparkling" in row["Variety"]:
        complete_df.loc[index,"Type"] = "Sparkling"
    elif "Champagne" in row["Variety"]:
        complete_df.loc[index,"Type"] = "Sparkling"
    elif "Prosecco" in row["Variety"]:
        complete_df.loc[index,"Type"] = "Sparkling"
    elif "Lambrusco" in row["Variety"]:
        complete_df.loc[index,"Type"] = "Sparkling" 
    #red wines
    elif "Red" in row["Variety"]:
        complete_df.loc[index,"Type"] = "Red"
    elif "Noir" in row["Variety"]:
        complete_df.loc[index,"Type"] = "Red"
    elif "Tint" in row["Variety"]:
        complete_df.loc[index,"Type"] = "Red"
    elif "Cabernet" in row["Variety"]:
        complete_df.loc[index,"Type"] = "Red"
    elif "Syrah" in row["Variety"]:
        complete_df.loc[index,"Type"] = "Red"
    elif "Sirah" in row["Variety"]:
        complete_df.loc[index,"Type"] = "Red"
    elif "Malbec" in row["Variety"]:
        complete_df.loc[index,"Type"] = "Red"
    elif "Gamay" in row["Variety"]:
        complete_df.loc[index,"Type"] = "Red"
    elif "Merlot" in row["Variety"]:
        complete_df.loc[index,"Type"] = "Red"
    elif "Barbera" in row["Variety"]:
        complete_df.loc[index,"Type"] = "Red"
    elif "Shiraz" in row["Variety"]:
        complete_df.loc[index,"Type"] = "Red"
    elif row["Variety"] == "Port":
        complete_df.loc[index,"Type"] = "Red"
    elif "Nero" in row["Variety"]:
        complete_df.loc[index,"Type"] = "Red"        
    elif "Negro" in row["Variety"]:
        complete_df.loc[index,"Type"] = "Red"   
    elif "Sangiovese" in row["Variety"]:
        complete_df.loc[index,"Type"] = "Red"        
    elif row["Variety"] == "Grenache":
        complete_df.loc[index,"Type"] = "Red"     
    elif row["Variety"] == "Tempranillo":
        complete_df.loc[index,"Type"] = "Red" 
    elif "Nebbiolo" in row["Variety"]:
        complete_df.loc[index,"Type"] = "Red"        
#White wines        
    elif "White" in row["Variety"]:
        complete_df.loc[index,"Type"] = "White"
    elif "Chardonnay" in row["Variety"]:
        complete_df.loc[index,"Type"] = "White"
    elif "Riesling" in row["Variety"]:
        complete_df.loc[index,"Type"] = "White"
    elif "Grigio" in row["Variety"]:
        complete_df.loc[index,"Type"] = "White"
    elif "Gris" in row["Variety"]:
        complete_df.loc[index,"Type"] = "White"
    elif "Blanc" in row["Variety"]:
        complete_df.loc[index,"Type"] = "White"
    elif "Gewürztraminer" in row["Variety"]:
        complete_df.loc[index,"Type"] = "White"
    elif "Moscato" in row["Variety"]:
        complete_df.loc[index,"Type"] = "White"
    elif "Musca" in row["Variety"]:
        complete_df.loc[index,"Type"] = "White"
    elif "Semillon" in row["Variety"]:
        complete_df.loc[index,"Type"] = "White"
    elif "Viognier" in row["Variety"]:
        complete_df.loc[index,"Type"] = "White"
    elif "Cortese" in row["Variety"]:
        complete_df.loc[index,"Type"] = "White"
    elif "Bianco" in row["Variety"]:
        complete_df.loc[index,"Type"] = "White"
      
    else:
        complete_df.loc[index,"Type"] = "Other"
 

In [19]:
complete_df.to_csv('winedata_coord_type.csv')

In [20]:
complete_df

,Unnamed: 0,Title,Variety,Year,Price,Grade,Winery,Province,Region,Country,Taster,Taster_Twitter,Description,Location,Latitude,Longitude,Type
0,1,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,2011,15.0,87,Quinta dos Avidagos,Douro,NaN,Portugal,Roger Voss,@vossroger,"This is ripe and fruity, a wine that is smooth...",Douro Portugal,41.512,-5.49225,Red
1,608,Quinta do Portal 2012 Verdelho and Sauvignon B...,Portuguese White,2012,15.0,87,Quinta do Portal,Douro,NaN,Portugal,Roger Voss,@vossroger,Verdelho and Sauvignon Blanc hangs together ea...,Douro Portugal,41.512,-5.49225,White
2,647,Quinta do Monte Xisto 2013 Red (Douro),Portuguese Red,2013,80.0,92,Quinta do Monte Xisto,Douro,NaN,Portugal,Roger Voss,@vossroger,"This is an impressive, powerful wine. It speak...",Douro Portugal,41.512,-5.49225,Red
3,784,Quinta do Sagrado 2008 VT '08 Red (Douro),Portuguese Red,2008,32.0,90,Quinta do Sagrado,Douro,NaN,Portugal,Roger Voss,@vossroger,"This mature wine has a soft, smooth character ...",Douro Portugal,41.512,-5.49225,Red
4,786,Sobredos 2012 Aneto Tinto Red (Douro),Portuguese Red,2012,12.0,90,Sobredos,Douro,NaN,Portugal,Roger Voss,@vossroger,"A ripe and structured wine, this has bold blac...",Douro Portugal,41.512,-5.49225,Red
5,874,Quinta do Vallado 2009 Sousão (Douro),Sousão,2009,40.0,90,Quinta do Vallado,Douro,NaN,Portugal,Roger Voss,@vossroger,"Sousão gives intense color and that, along wit...",Douro Portugal,41.512,-5.49225,Other
6,887,Maritávora 2009 Reserva Branco White (Douro),Portuguese White,2009,60.0,90,Maritávora,Douro,NaN,Portugal,Roger Voss,@vossroger,Aromas of fresh-cut hay lead to a wine that ha...,Douro Portugal,41.512,-5.49225,White
7,901,Quinta do Vallado 2010 Reserva Branco White (D...,Portuguese White,2010,50.0,90,Quinta do Vallado,Douro,NaN,Portugal,Roger Voss,@vossroger,"A wood-aged white, spicy and ripe. It has just...",Douro Portugal,41.512,-5.49225,White
8,947,Real Companhia Velha 2014 Evel Tinto Red (Douro),Portuguese Red,2014,13.0,87,Real Companhia Velha,Douro,NaN,Portugal,Roger Voss,@vossroger,Just lightly touched by wood this is a bright ...,Douro Portugal,41.512,-5.49225,Red
9,1072,Poças 2014 Símbolo Red (Douro),Portuguese Red,2014,150.0,94,Poças,Douro,NaN,Portugal,Roger Voss,@vossroger,This is a major Douro wine from the Poças fami...,Douro Portugal,41.512,-5.49225,Red
